In [ ]:
from ipynb.fs.full.common import *
# %run common.ipynb

Variables

In [ ]:
# # ['Hemlata', 'Malti', 'Preeti', 'Sharifa', 'Vinita', 'VKS'] ['Geeta', 'Jitendra', 'Jyoti', 'Kuldeep', 'Seema', 'VijayLaxmi']
# ACTIVE_SHAM = 'Active'
# SAMPLE = 'Hemlata'
# ELECTRODES = '32electrodes' # '32electrodes' 
GROUP1 = 'Pre'
GROUP2 = 'Post' # generally GROUP1+1
mapping = {0: 'theta', 1: 'alpha', 2: 'beta'}

## LOAD DATA

In [ ]:
folder_path = os.path.join(os.getcwd(), 'Depression-Sample-dataset-AIIMS', ELECTRODES, ACTIVE_SHAM)
all_files = []

folders = ['2Pre-TDCS', '4Post-TDCS']
for folder in folders:
    ext = 'Close.easy' if ELECTRODES == '32electrodes' else 'EEG.easy'
    for _ in glob.glob(os.path.join(folder_path, folder, SAMPLE) + '/*' + ext):
        all_files.append(_)

raw_1 = data_transformation_easy(all_files[0])
raw_1 = g1_preprocess(raw_1)
raw_2 = data_transformation_easy(all_files[1])
raw_2 = g1_preprocess(raw_2)

Excel data

In [ ]:
existing_file = f"comparison-results/result-active-latest.xlsx"
df = pd.read_excel(existing_file, header=[0,1])

main_column_names = ['index', 'swn', 'cc', 'lavg', 'eglo', 'eloc']
subcols = ('theta-g1', 'theta-g2', 'alpha-g1', 'alpha-g2', 'beta-g1', 'beta-g2')

# subcolumn_names = {
#     'index' : ['index'],
#     # 'links': subcols,
#     # 'asymmetry': subcols,
#     'swn': subcols,
#     'cc' : subcols,
#     'lavg': subcols,
#     'eglo': subcols,
#     'eloc': subcols,
#     # 'hubs': ('theta', 'alpha', 'beta'),
#     # 'channels': ('theta', 'alpha', 'beta'),
#     # 'regions': ('theta', 'alpha', 'beta'),
# }

# columns_tuples = [(main_col, sub_col) for main_col in main_column_names for sub_col in subcolumn_names[main_col]]
# columns = pd.MultiIndex.from_tuples(columns_tuples)
# df = pd.DataFrame(columns=columns)

row_data = {('index', 'index'): f'{ACTIVE_SHAM}-{SAMPLE}-{GROUP1}vs{GROUP2}'}
df

### 1. PSD graph plot between MDD and Control group

For resting state EEG channel, analyzing the overall average power across channels provides a holistic view of the brain's activity without emphasizing the specificity of individual channels.

1.1 PSD using plot_psd

In [ ]:
fig, ax = plt.subplots()
#plot_psd uses welch method for continuous data
raw_1.plot_psd(picks=raw_1.info['ch_names'], average=True, fmin=FMIN, fmax=FMAX, ax=ax, show=False, color='blue', dB=False)
raw_2.plot_psd(picks=raw_2.info['ch_names'], average=True, fmin=FMIN, fmax=FMAX, ax=ax, show=False, color='red', dB=False)

# dB = True plots PSD in decibels (logarithmic)
# different n_fft compared to psd_array_welch
ax.set_xlabel('Frequency (Hz)')
ax.set_ylabel('Power/Frequency (microV^2/Hz)')
ax.set_title('Power Spectral Density Comparison')

ax.text(0.8, 0.9, GROUP1, color='blue', transform=ax.transAxes)
ax.text(0.8, 0.85, GROUP2, color='red', transform=ax.transAxes)

plt.ylim(0, 8)
# plt.show()

fig.savefig(f'comparison-results/PSD - {SAMPLE} - {GROUP1} vs {GROUP2}.png')

### 2. Epoching

In [ ]:
duration = 4.0
overlap = 2.0 

samples_per_epoch = int(duration * SFREQ)
samples_per_overlap = int(overlap * SFREQ)

# Manually created events
start, stop = 0, samples_per_epoch
events = []
while stop <= len(raw_1):
    events.append([start, 0, 1]) 
    start += samples_per_overlap
    stop += samples_per_overlap

events = np.array(events)

2.1 Segmenting epochs for Group 1

In [ ]:
epochs_1 = mne.Epochs(raw_1, events, tmin=0, tmax=duration, baseline=None, detrend=1,
                    picks=None, preload=True, reject=None)
# raw_1.plot(n_channels=len(raw_1.info['ch_names']), events=events, event_color={1:'r'}, scalings=SCALINGS)
# epochs_1.plot(n_channels=len(raw_1.info['ch_names']), event_color={1:'r'}, events=events, scalings=SCALINGS)

In [ ]:
epochs_2 = mne.Epochs(raw_2, events, tmin=0, tmax=duration, baseline=None, detrend=1,
                    picks=None, preload=True, reject=None)
# raw_2.plot(n_channels=len(raw_1.info['ch_names']), events=events, event_color={1:'r'}, scalings=SCALINGS)
# epochs_2.plot(n_channels=len(raw_1.info['ch_names']), event_color={1:'r'}, events=events, scalings=SCALINGS)

### 3. PLI and construction of brain function matrices

In [ ]:
# pli method always gives positive correlations
connectivity = {}
def connectivity_matrix(epochs, i):
    return spectral_connectivity_epochs(
    epochs, method='pli', mode='multitaper', sfreq=SFREQ,
    fmin=FREQ_BANDS[mapping[i]][0], fmax=FREQ_BANDS[mapping[i]][1], faverage=True, n_jobs=1)


In [ ]:
n_channels = len(raw_1.info['ch_names'])
for i in mapping:
    con = connectivity_matrix(epochs_1, i)
    connectivity[f'{GROUP1}-{mapping[i]}'] = con.get_data().reshape((n_channels, n_channels))

    con = connectivity_matrix(epochs_2, i)
    connectivity[f'{GROUP2}-{mapping[i]}'] = con.get_data().reshape((n_channels, n_channels))


### 3. Thresholding - M1 -  Small World Index

Preserves small-world properties in both groups to ensure that any observed differences in connectivity are not biased by the thresholding method.
Preserving these properties ensures that information can be transmitted quickly and effectively across the network.

In [ ]:
def make_symmetric(arr, n_channels):
    for row in range(n_channels):
        for col in range(n_channels):
            arr[row][col] = arr[col][row]
    return arr

In [ ]:
def calculate_swn(mat, threshold):
    tthresh = np.median(mat[mat!=0]) + threshold*np.std(mat[mat!=0])
    binmat= mat > tthresh
    n_connections = np.count_nonzero(binmat == 1)
    binmat = make_symmetric(binmat.astype(int), n_channels)
    g= calculate_avergae_components(nx.from_numpy_array(binmat))
    Lw_binarized, CC_binarized = g[0], g[3]
    return (CC_binarized/Lw_binarized if n_connections else 0), n_connections

In [ ]:
n_perms = 1000
n_network = n_perms//10
swn_z_1, swn_z_2 = {}, {}
optimal_swn_perms, optimal_swn_real1, optimal_swn_real2 = {}, {}, {}

for i in mapping:
    conmat1, conmat2 = connectivity[f'{GROUP1}-{mapping[i]}'], connectivity[f'{GROUP2}-{mapping[i]}']
    if ACTIVE_SHAM == 'Active':
        x = {'theta': 0.9422110552763826, 'alpha': 0.4824120603015075, 'beta': 1.2738693467336675}
    else:
        x = {'theta': 0.2788944723618091, 'alpha': 0.391959798994975, 'beta': 0.2638190954773868}
    threshold = x[mapping[i]]

### 4. Binarization of functional connectivity

In [ ]:
# Find optimal threhsold
binarized_matrix = {}
print(x[mapping[i]])
for i in mapping:
    conmat1, conmat2 = connectivity[f'{GROUP1}-{mapping[i]}'], connectivity[f'{GROUP2}-{mapping[i]}']
    tthresh1 = np.median(conmat1[conmat1!=0]) + x[mapping[i]]*np.std(conmat1[conmat1!=0])
    tthresh2 = np.median(conmat2[conmat2!=0]) + x[mapping[i]]*np.std(conmat2[conmat2!=0])
    binarized_matrix[f'{GROUP1}-{mapping[i]}'] = connectivity[f'{GROUP1}-{mapping[i]}'] > tthresh1
    binarized_matrix[f'{GROUP2}-{mapping[i]}'] = connectivity[f'{GROUP2}-{mapping[i]}'] > tthresh2
    print(tthresh1, tthresh2)
print(binarized_matrix.keys())


In [ ]:
selected_regions = {}
selected_regions['theta'] = ['LF', 'RC', 'LPO']
selected_regions['alpha'] = ['LF', 'RC', 'LPO']
selected_regions['beta'] = ['RF', 'LF', 'LC']
selected_regions

ADD EXTRAS IN EXCEL FILE

In [ ]:
regionvschannel = {
    'LC': ['C3', 'CP1', 'CP5', 'FC1', 'FC5'], # Left central 
    'LF': ['Fp1', 'AF3', 'F3', 'F7'], # Left frontal
    'LT': ['T7'], # Left temporal
    'LPO': ['PO3',  'P3', 'P7', 'O1'], # Left parietal-occipital
    'RC': ['CP6', 'FC6', 'C4', 'FC2', 'CP2',], # Right central 
    'RF': ['F8', 'F4', 'AF4', 'Fp2'], # Right frontal
    'RT': ['T8'], # Right temporal
    'RPO': ['P8', 'P4', 'PO4', 'O2'] , # Right parietal-occipital
}

In [ ]:
for i in mapping:
    nodes = list(itertools.chain(*[regionvschannel[region] for region in selected_regions[mapping[i]]]))
    node_indexes = {raw_1.info['ch_names'].index(_) for _ in nodes}

    if len(node_indexes):
        g1 = calculate_avergae_components(nx.from_numpy_array(binarized_matrix[f'{GROUP1}-{mapping[i]}']).subgraph(node_indexes))
        g2 = calculate_avergae_components(nx.from_numpy_array(binarized_matrix[f'{GROUP2}-{mapping[i]}']).subgraph(node_indexes))


        row_data[('swn', f'{mapping[i]}-g1')] =  g1[3]/g1[0] if g1[0] and g2[0] else 0
        row_data[('swn', f'{mapping[i]}-g2')] =  g2[3]/g2[0] if g1[0] and g2[0] else 0
        row_data[('cc', f'{mapping[i]}-g1')] = g1[3]
        row_data[('cc', f'{mapping[i]}-g2')] = g2[3]
        row_data[('lavg', f'{mapping[i]}-g1')] = g1[0]
        row_data[('lavg', f'{mapping[i]}-g2')] = g2[0]
        row_data[('nbc', f'{mapping[i]}-g1')] = g1[1]
        row_data[('nbc', f'{mapping[i]}-g2')] = g2[1]
        row_data[('eglo', f'{mapping[i]}-g1')] = g1[2]
        row_data[('eglo', f'{mapping[i]}-g2')] = g2[2]
        row_data[('eloc', f'{mapping[i]}-g1')] = g1[4]
        row_data[('eloc', f'{mapping[i]}-g2')] = g2[4]
    else:
        row_data[('swn', f'{mapping[i]}-g1')] =  0
        row_data[('swn', f'{mapping[i]}-g2')] = 0
        row_data[('cc', f'{mapping[i]}-g1')] = 0
        row_data[('cc', f'{mapping[i]}-g2')] = 0
        row_data[('lavg', f'{mapping[i]}-g1')] = 0
        row_data[('lavg', f'{mapping[i]}-g2')] = 0
        row_data[('nbc', f'{mapping[i]}-g1')] = 0
        row_data[('nbc', f'{mapping[i]}-g2')] = 0
        row_data[('eglo', f'{mapping[i]}-g1')] = 0
        row_data[('eglo', f'{mapping[i]}-g2')] = 0
        row_data[('eloc', f'{mapping[i]}-g1')] = 0
        row_data[('eloc', f'{mapping[i]}-g2')] = 0

In [ ]:
new_row = pd.Series(row_data)
df.loc[len(df)] = new_row

In [ ]:
df.to_excel(existing_file, index=True, header=True)
df